[(토론-3) 과연 mAP는 신뢰할만한 metric일까?](https://stages.ai/competitions/218/discussion/talk/post/1816)

Object Detection에서는 주로 mAP라는 metric을 활용해서 성능을 평가합니다. (1강 참조)

그렇다면 과연 mAP는 신뢰할만한 metric인지 고민해보면 좋을 것 같아 글을 작성하게 되었습니다.

우선 model의 output을 visualize하였을 때 mAP 점수에 따라서 어떠한 이미지가 나오게 될까요?

## 1. LB score : 0.5150

![](https://s3-ap-northeast-2.amazonaws.com/prod-aistages-public/app/Users/00000074/files/d7e2c968-5111-4bf9-9cfa-7406bed04092..png)

![](https://s3-ap-northeast-2.amazonaws.com/prod-aistages-public/app/Users/00000074/files/096b398c-7bbf-4a8b-baaf-f817676d0b6a..png)

![](https://s3-ap-northeast-2.amazonaws.com/prod-aistages-public/app/Users/00000074/files/16c9ef0b-b3b9-496f-8042-1f562108f27c..png)

![](https://s3-ap-northeast-2.amazonaws.com/prod-aistages-public/app/Users/00000074/files/31a459d6-2a03-461a-9b7a-6ded351aa08b..png)

## 2. LB score : 0.6068

![](https://s3-ap-northeast-2.amazonaws.com/prod-aistages-public/app/Users/00000074/files/bfc39b2b-f6c8-425b-8a20-67e2aad03dc6..png)

![](https://s3-ap-northeast-2.amazonaws.com/prod-aistages-public/app/Users/00000074/files/3df35f32-2993-4810-ad9f-4d6cf275b849..png)

![](https://s3-ap-northeast-2.amazonaws.com/prod-aistages-public/app/Users/00000074/files/ef620005-6214-4b39-b7fe-4984166e8499..png)

![](https://s3-ap-northeast-2.amazonaws.com/prod-aistages-public/app/Users/00000074/files/3900644b-446a-4470-9886-7fc153f7e985..png)

**왜 box가 더 많은 model의 output이 mAP성능이 높게 나올까요?**

정답은 바로 **PR curve**에 있습니다.

![](https://s3-ap-northeast-2.amazonaws.com/prod-aistages-public/app/Users/00000273/files/d8e68174-4622-4007-a25d-675f3597c486..png)

우선 mAP점수 산정방식을 보겠습니다. 위와 같이 PR curve를 구합니다. 이 때 recall과 precision은 **confidence score** 별로 각 점들이 생성됩니다.

그리고 아래의 넓이 (A+B)에 해당하는 부분이 AP가 됩니다. 이를 각 class 별로 계산하여 mAP를 구하게 됩니다.

다시 사진을 다시 보겠습니다.

![](https://s3-ap-northeast-2.amazonaws.com/prod-aistages-public/app/Users/00000273/files/5f8f64a2-4cd0-411c-bc8e-331fb2c875c7..png)

![](https://s3-ap-northeast-2.amazonaws.com/prod-aistages-public/app/Users/00000273/files/1fb79d2c-50a3-4742-bd8f-e256d4b38033..png)

위의 두 사진을 비교하였을 때 확실히 1번 model이 더 잘 detect 된 것으로 보입니다.

**그렇다면 2번 model은 물체를 탐지하지 못했을까요?** 아닙니다. 수많은 box들 중 정답 box도 존재할 것 입니다. 그렇다면 저 많은 box들이 mAP를 올리는데 기여한 것 입니다.

**그렇다면 왜 mAP가 오를까요?** PR curve의 recall과 precision은 긍정 데이터에 대한 예측률입니다. 그래프를 보시면 낮은 threshold에서 precision이 떨어지더라도 recall이 조금이라도 오른다면 graph의 면적을 넓어지게 됩니다. 즉 mAP가 오르는 것입니다. 아래의 box친 부분이 넓어진 면적으로 어림짐작 할 수 있습니다.

![](https://s3-ap-northeast-2.amazonaws.com/prod-aistages-public/app/Users/00000273/files/9b560376-ebf8-43d0-a415-3b916e22e6db..png)

**그렇다면 무작정 box를 많이 생성하면 안되나요?** 그렇게 되면 높은 threshold 에서의 precision도 떨어져 mAP성능 또한 떨어지게 됩니다. 그렇기 때문에 낮은 threshold의 box가 object 근처에 있을 법한 box가 생성되는 것이 중요합니다.

**그렇다면 실제 서비스화를 할 때 어떻게 하나요?** 실제 서비스화 할 때는 threshold를 적절히 설정하여 서빙하게 됩니다. 또한 위의 사진처럼 visualize하지 않고 confidence score 별로 box 색을 진하게 생성하게 된다면 threshold 별로 output이 어떻게 나오는지 알 수 있습니다.

위와 같은 이유로 mAP의 성능이 결정이 되며, 주로 mAP가 낮은 model의 경우 object를 탐지하더라도 낮은 score를 가지고 있으며 어려운 물체는 탐지조차 못합니다. mAP가 높은 model의 경우 object를 탐지할 때 높은 score를 가지고 있으며 어려운 물체를 낮은 score에서라도 탐지하는 것으로 보입니다.

이렇듯 mAP와 실제 output에 대해 알아보았는데 여러분들의 의견은 어떤지 남겨주세요 🙂

관련 metric에 관한 글도 공유드립니다.

- (링크) [VinBigData Chest X-ray Abnormalities Detection](https://www.kaggle.com/c/vinbigdata-chest-xray-abnormalities-detection/discussion/229637)